# 🧬 HBV Tahoe-x1 분석 - 실제 데이터 (메모리 효율 버전)

**목적**: GSE182159 대용량 데이터에 Tahoe-x1 적용 (메모리 최적화)

---

## ⚠️ 메모리 관리 전략

| 전략 | 설명 |
|------|------|
| **샘플링** | 전체 243K 중 일부만 분석 (옵션) |
| **Sparse Matrix** | 메모리 효율적 형식 유지 |
| **배치 처리** | 임베딩을 작은 배치로 추출 |
| **즉시 삭제** | 불필요한 변수 즉시 삭제 |

In [ ]:
# @title STEP 1: 환경 설정
import os, sys, platform

# Colab 확인
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/tahoe-x1'
else:
    PROJECT_PATH = 'G:/내 드라이브/tahoe-x1'

DATA_PATH = f'{PROJECT_PATH}/viral_hepatitis/raw_data'
OUTPUT_PATH = f'{PROJECT_PATH}/viral_hepatitis/analysis_results'
os.makedirs(OUTPUT_PATH, exist_ok=True)

if IN_COLAB:
    os.chdir(PROJECT_PATH)
    sys.path.insert(0, PROJECT_PATH)
    print('📦 패키지 설치 중...')
    !pip install scanpy anndata einops omegaconf igraph leidenalg boto3 mosaicml transformers -q
    !pip install -e . -q

import gc
print(f'\n✅ STEP 1 완료!')
print(f'📁 데이터: {DATA_PATH}')
print(f'📁 결과: {OUTPUT_PATH}')

In [ ]:
# @title STEP 2: 데이터 로드 (메모리 효율적)
import scanpy as sc
import anndata as ad
import numpy as np
import scipy.sparse as sp

DATA_FILE = 'GSE182159_processed.h5ad'

# ========== 샘플링 옵션 ==========
USE_SAMPLING = True  # @param {type:"boolean"}
SAMPLE_SIZE = 50000  # @param {type:"integer"}
# =================================

print(f'📊 데이터 로딩: {DATA_FILE}')

if USE_SAMPLING:
    print(f'⚡ 샘플링 모드: {SAMPLE_SIZE:,}개 세포만 사용')
    
    # backed 모드로 열기 (메모리에 전체 로드 안함)
    adata_full = ad.read_h5ad(f'{DATA_PATH}/{DATA_FILE}', backed='r')
    print(f'   전체 데이터: {adata_full.n_obs:,} cells')
    
    # 랜덤 샘플링 (HBV stage 비율 유지)
    np.random.seed(42)
    if adata_full.n_obs > SAMPLE_SIZE:
        indices = np.random.choice(adata_full.n_obs, SAMPLE_SIZE, replace=False)
        indices = np.sort(indices)
    else:
        indices = np.arange(adata_full.n_obs)
    
    # 서브셋 추출 (메모리에 로드)
    adata = adata_full[indices].to_memory()
    del adata_full
    gc.collect()
else:
    print('⚠️ 전체 데이터 로드 (메모리 주의!)')
    adata = ad.read_h5ad(f'{DATA_PATH}/{DATA_FILE}')

print(f'\n✅ STEP 2 완료!')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   유전자 수: {adata.n_vars:,}')
print(f'\n📊 HBV Stage 분포:')
if 'hbv_stage' in adata.obs.columns:
    print(adata.obs['hbv_stage'].value_counts())
elif 'Stage' in adata.obs.columns:
    print(adata.obs['Stage'].value_counts())

In [ ]:
# @title STEP 3: Tahoe-x1 모델 로드
import torch
from tqdm.auto import tqdm
from datetime import datetime

from tahoe_x1.model import ComposerTX
from tahoe_x1.utils.util import loader_from_adata

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_SIZE = '70m'

print(f'🖥️ Device: {DEVICE}')
print(f'🚀 Tahoe-x1 {MODEL_SIZE} 모델 로딩 중...')

model, vocab, model_cfg, collator_cfg = ComposerTX.from_hf(
    repo_id='tahoebio/Tahoe-x1',
    model_size=MODEL_SIZE,
    attn_impl='torch'
)
model.to(DEVICE).eval()

if DEVICE == 'cuda':
    torch.cuda.empty_cache()

print(f'\n✅ STEP 3 완료!')

In [ ]:
# @title STEP 4: 데이터 전처리 (메모리 효율적)
print('🧬 데이터 전처리 시작...')

# 원본 메타데이터 보존
obs_backup = adata.obs.copy()

# 기본 필터링 (in-place)
sc.pp.filter_genes(adata, min_cells=3)
print(f'   필터링 후: {adata.n_vars:,} genes')

# id_in_vocab 추가 (벡터화된 방식)
print('   Vocab 매핑 중...')
gene_names = adata.var_names.tolist()
vocab_ids = np.array([vocab.token_to_index.get(g, -1) for g in gene_names])
adata.var['id_in_vocab'] = vocab_ids

# 유효한 유전자만 선택
valid_mask = vocab_ids >= 0
print(f'   Vocab 매핑: {valid_mask.sum():,} / {len(valid_mask):,} genes')

adata = adata[:, valid_mask].copy()

# 메모리 정리
gc.collect()

print(f'\n✅ STEP 4 완료!')
print(f'   전처리 후: {adata.shape}')

In [ ]:
# @title STEP 5: 임베딩 추출 (배치 처리)
BATCH_SIZE = 4  # 메모리 절약을 위해 작은 배치

print(f'🧠 임베딩 추출 (배치: {BATCH_SIZE})')
print(f'   세포 수: {adata.n_obs:,}')

loader = loader_from_adata(
    adata=adata,
    collator_cfg=collator_cfg,
    vocab=vocab,
    batch_size=BATCH_SIZE,
    num_workers=2,
    prefetch_factor=2
)

all_embeddings = []
with torch.no_grad():
    for batch in tqdm(loader, desc='추출 중'):
        batch = {k: v.to(DEVICE) if isinstance(v, torch.Tensor) else v 
                 for k, v in batch.items()}
        output = model(batch, skip_decoders=True)
        all_embeddings.append(output['cell_emb'].cpu().numpy())
        
        # 메모리 정리
        del output, batch
        if DEVICE == 'cuda':
            torch.cuda.empty_cache()

embeddings = np.vstack(all_embeddings)
del all_embeddings
gc.collect()

adata.obsm[f'X_tahoe_{MODEL_SIZE}'] = embeddings

print(f'\n✅ STEP 5 완료!')
print(f'   임베딩 차원: {embeddings.shape}')

In [ ]:
# @title STEP 6: UMAP 및 클러스터링
import matplotlib.pyplot as plt

print('🗺️ UMAP 및 클러스터링...')

sc.pp.neighbors(adata, use_rep=f'X_tahoe_{MODEL_SIZE}')
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sc.pl.umap(adata, color='leiden', ax=axes[0], show=False,
           title='Cell Clusters', legend_loc='on data')

if 'tissue' in adata.obs.columns:
    sc.pl.umap(adata, color='tissue', ax=axes[1], show=False,
               title='Tissue (Blood vs Liver)')

stage_col = 'hbv_stage' if 'hbv_stage' in adata.obs.columns else 'Stage'
if stage_col in adata.obs.columns:
    sc.pl.umap(adata, color=stage_col, ax=axes[2], show=False,
               title='HBV Stage')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/hbv_real_umap_overview.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 6 완료!')

In [ ]:
# @title STEP 7: 세포 유형 식별
print('🔬 세포 유형 분석 중...\n')

# PBMC + Liver 마커
markers = {
    'CD4+ T cells': ['CD3D', 'CD4', 'IL7R'],
    'CD8+ T cells': ['CD3D', 'CD8A', 'CD8B'],
    'B cells': ['MS4A1', 'CD79A', 'CD79B'],
    'NK cells': ['GNLY', 'NKG7'],
    'CD14+ Monocytes': ['CD14', 'LYZ', 'S100A8'],
    'FCGR3A+ Monocytes': ['FCGR3A', 'MS4A7'],
    'Dendritic cells': ['FCER1A', 'CST3'],
    'Platelets': ['PPBP', 'PF4'],
    'Hepatocytes': ['ALB', 'APOA1', 'HP'],
    'Kupffer cells': ['CD68', 'MARCO'],
    'Endothelial': ['PECAM1', 'VWF']
}

# 클러스터별 세포 유형
cluster_types = {}
for cluster in sorted(adata.obs['leiden'].unique(), key=lambda x: int(x)):
    cluster_cells = adata[adata.obs['leiden'] == cluster]
    best_type, best_score = 'Unknown', 0
    
    for cell_type, genes in markers.items():
        valid_genes = [g for g in genes if g in adata.var_names]
        if valid_genes:
            X_subset = cluster_cells[:, valid_genes].X
            if sp.issparse(X_subset):
                score = X_subset.toarray().mean()
            else:
                score = X_subset.mean()
            if score > best_score:
                best_score, best_type = score, cell_type
    
    cluster_types[cluster] = best_type
    print(f'  클러스터 {cluster:>2}: {best_type:<20} ({len(cluster_cells):>5}개 세포)')

adata.obs['cell_type'] = adata.obs['leiden'].map(cluster_types)

# 시각화
fig, ax = plt.subplots(figsize=(12, 10))
sc.pl.umap(adata, color='cell_type', ax=ax, show=False, title='Cell Types')
plt.savefig(f'{OUTPUT_PATH}/hbv_real_cell_types.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 7 완료!')

In [ ]:
# @title STEP 8: 결과 저장
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
sample_tag = f'_{SAMPLE_SIZE//1000}k' if USE_SAMPLING else '_full'
output_file = f'{OUTPUT_PATH}/hbv_real_tahoe_analysis{sample_tag}_{timestamp}.h5ad'

adata.write_h5ad(output_file)

print('=' * 60)
print('🎉 GSE182159 실제 데이터 Tahoe-x1 분석 완료!')
print('=' * 60)
print(f'\n📁 저장된 파일:')
print(f'   📊 {output_file}')
print(f'   🖼️ {OUTPUT_PATH}/hbv_real_umap_overview.png')
print(f'   🖼️ {OUTPUT_PATH}/hbv_real_cell_types.png')
print(f'\n📋 분석 요약:')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   클러스터 수: {adata.obs["leiden"].nunique()}')
print(f'\n📊 조직별 세포 수:')
print(adata.obs['tissue'].value_counts())
print('\n✅ 다음 단계: Step2_HBV_group_comparison.ipynb 실행')